<a href="https://colab.research.google.com/github/clairewangqc/ml_projects/blob/main/hotel_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd


In [3]:
data = pd.read_csv('https://raw.githubusercontent.com/cjflanagan/cs68/master/Datafiniti_Hotel_Reviews%20-%20Datafiniti_Hotel_Reviews.csv')

In [4]:
data.head()

,id,dateAdded,dateUpdated,address,categories,primaryCategories,city,country,keys,latitude,longitude,name,postalCode,province,reviews.date,reviews.dateSeen,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username,sourceURLs,websites
0,AVwc252WIN2L1WUfpqLP,2016-10-30T21:42:42Z,2018-09-10T21:06:27Z,5921 Valencia Cir,"Hotels,Hotels and motels,Hotel and motel reser...",Accommodation & Food Services,Rancho Santa Fe,US,us/ca/ranchosantafe/5921valenciacir/359754519,32.990959,-117.186136,Rancho Valencia Resort Spa,92067,CA,2013-11-14T00:00:00Z,"2016-08-03T00:00:00Z,2016-07-26T00:00:00Z,2016...",5.0,https://www.hotels.com/hotel/125419/reviews%20/,Our experience at Rancho Valencia was absolute...,Best romantic vacation ever!!!!,NaN,NaN,Paula,http://www.hotels.com/ho125419/%25252525253Flo...,http://www.ranchovalencia.com
1,AVwc252WIN2L1WUfpqLP,2016-10-30T21:42:42Z,2018-09-10T21:06:27Z,5921 Valencia Cir,"Hotels,Hotels and motels,Hotel and motel reser...",Accommodation & Food Services,Rancho Santa Fe,US,us/ca/ranchosantafe/5921valenciacir/359754519,32.990959,-117.186136,Rancho Valencia Resort Spa,92067,CA,2014-07-06T00:00:00Z,"2016-08-02T00:00:00Z,2016-08-26T00:00:00Z,2016...",5.0,https://www.hotels.com/hotel/125419/reviews%20/,Amazing place. Everyone was extremely warm and...,Sweet sweet serenity,NaN,NaN,D,http://www.hotels.com/ho125419/%25252525253Flo...,http://www.ranchovalencia.com
2,AVwc252WIN2L1WUfpqLP,2016-10-30T21:42:42Z,2018-09-10T21:06:27Z,5921 Valencia Cir,"Hotels,Hotels and motels,Hotel and motel reser...",Accommodation & Food Services,Rancho Santa Fe,US,us/ca/ranchosantafe/5921valenciacir/359754519,32.990959,-117.186136,Rancho Valencia Resort Spa,92067,CA,2015-01-02T00:00:00Z,"2016-11-15T00:00:00Z,2016-08-23T00:00:00Z,2016...",5.0,https://www.hotels.com/hotel/125419/reviews%20/,We booked a 3 night stay at Rancho Valencia to...,Amazing Property and Experience,NaN,NaN,Ron,http://www.hotels.com/ho125419/%25252525253Flo...,http://www.ranchovalencia.com
3,AVwdOclqIN2L1WUfti38,2015-11-28T19:19:35Z,2018-09-10T21:06:16Z,7520 Teague Rd,"Hotels,Hotels and motels,Travel agencies and b...",Accommodation & Food Services,Hanover,US,us/md/hanover/7520teaguerd/-2043779672,39.155929,-76.716341,Aloft Arundel Mills,21076,MD,2016-05-15T00:00:00Z,"2016-05-21T00:00:00Z,2016-07-31T00:00:00Z",2.0,https://www.tripadvisor.com/Hotel_Review-g4118...,Currently in bed writing this for the past hr ...,"Never again...beware, if you want sleep.",Richmond,VA,jaeem2016,http://www.yellowbook.com/profile/aloft-arunde...,http://www.starwoodhotels.com/alofthotels/prop...
4,AVwdOclqIN2L1WUfti38,2015-11-28T19:19:35Z,2018-09-10T21:06:16Z,7520 Teague Rd,"Hotels,Hotels and motels,Travel agencies and b...",Accommodation & Food Services,Hanover,US,us/md/hanover/7520teaguerd/-2043779672,39.155929,-76.716341,Aloft Arundel Mills,21076,MD,2016-07-09T00:00:00Z,2016-07-31T00:00:00Z,5.0,https://www.tripadvisor.com/Hotel_Review-g4118...,I live in Md and the Aloft is my Home away fro...,ALWAYS GREAT STAY...,Laurel,MD,MamaNiaOne,http://www.yellowbook.com/profile/aloft-arunde...,http://www.starwoodhotels.com/alofthotels/prop...


In [5]:
data['reviews.text']

0       Our experience at Rancho Valencia was absolute...
1       Amazing place. Everyone was extremely warm and...
2       We booked a 3 night stay at Rancho Valencia to...
3       Currently in bed writing this for the past hr ...
4       I live in Md and the Aloft is my Home away fro...
                              ...                        
3964                           Nice hotel/ great location
3965    very nice stay.. will be back to another location
3966    Only stayed one night but that was enough to e...
3967    I have been staying at the Four Points Sherato...
3968    Thank you Erik for taking care of my friends a...
Name: reviews.text, Length: 3969, dtype: object

## **Examing the dataset**

In [6]:
data.shape #(3969, 25) 
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3969 entries, 0 to 3968
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    3969 non-null   object 
 1   dateAdded             3969 non-null   object 
 2   dateUpdated           3969 non-null   object 
 3   address               3969 non-null   object 
 4   categories            3969 non-null   object 
 5   primaryCategories     3969 non-null   object 
 6   city                  3969 non-null   object 
 7   country               3969 non-null   object 
 8   keys                  3969 non-null   object 
 9   latitude              3969 non-null   float64
 10  longitude             3969 non-null   float64
 11  name                  3969 non-null   object 
 12  postalCode            3969 non-null   object 
 13  province              3969 non-null   object 
 14  reviews.date          3969 non-null   object 
 15  reviews.dateSeen     

In [7]:
data['reviews.text'].head(10)

0    Our experience at Rancho Valencia was absolute...
1    Amazing place. Everyone was extremely warm and...
2    We booked a 3 night stay at Rancho Valencia to...
3    Currently in bed writing this for the past hr ...
4    I live in Md and the Aloft is my Home away fro...
5    I stayed here with my family for my daughters ...
6    Beautiful rooms and the nicest people working ...
7    We stayed here while visiting Maryland Live!. ...
8    I travel a lot with my job, so I'm constantly ...
9    In my line of work, I use meeting space in hot...
Name: reviews.text, dtype: object

In [8]:
#Print overviews of the first 10 reviews_text. 
for i in range(10): 
  print(data['reviews.text'].iloc[i])

Our experience at Rancho Valencia was absolutely perfect from beginning to end!!!! We felt special and very happy during our stayed. I would come back in a heart beat!!!
Amazing place. Everyone was extremely warm and welcoming. We've stayed at some top notch places and this is definitely in our top 2. Great for a romantic getaway or take the kids along as we did. Had a couple stuffed animals waiting for our girls upon arrival. Can't wait to go back.
We booked a 3 night stay at Rancho Valencia to play some tennis, since it is one of the highest rated tennis resorts in America. This place is really over the top from a luxury standpoint and overall experience. The villas are really perfect, the staff is great, attention to details (includes fresh squeezed orange juice each morning), restaurants, bar and room service amazing, and the tennis program was really impressive as well. We will want to come back here again.
Currently in bed writing this for the past hr 1/2 there have been dogs bar

In [9]:
#There are 307 different cities in the dataset. 
cities = data['city'].unique()
print(f'There are {len(cities)} different cities in the dataset. ')
print("")
print(cities) 

There are 307 different cities in the dataset. 

['Rancho Santa Fe' 'Hanover' 'Vancouver' 'Kansas City' 'Huntingdon'
 'Perry' 'San Francisco' 'Vineland' 'Oxon Hill' 'San Diego' 'Des Plaines'
 'Kissimmee' 'Chicago' 'Boston' 'New Orleans' 'Las Vegas' 'Miami Beach'
 'New York' 'Honolulu' 'Miami' 'Los Angeles' 'Beverly Hills' 'Coronado'
 'Philadelphia' 'Napa' 'Dallas' 'Key West' 'Seattle' 'Phoenix' 'Atlanta'
 'Anaheim' 'Atlantic City' 'La Jolla' 'Orlando' 'Concord' 'Shelbyville'
 'Woodbridge' 'Durham' 'Orange' 'Colorado Springs' 'Hawthorne' 'Henderson'
 'Fort Worth' 'San Antonio' 'Danville' 'Sedona' 'Acworth' 'Dodge City'
 'Baton Rouge' 'Flagstaff' 'New Castle' 'Cherokee' "Coeur d'Alene"
 'Middletown' 'Frisco' 'Arkansas City' 'Winfield' 'Dover' 'Gardner'
 'Ankeny' 'Brighton' 'Hilo' 'North Las Vegas' 'Alpharetta' 'East Tawas'
 'Northwood' 'Bloomingdale' 'Portland' 'Rockford' 'Austin'
 'Rancho Cucamonga' 'Auburn' 'Biddeford' 'Granbury' 'Williamsburg'
 'Aurora' 'Pico Rivera' 'Rosemead' 'El Se

In [10]:
#There are 462 different hotels in the dataset. 
hotels = data['name'].unique()
print(f'There are {len(hotels)} different hotels in the dataset. ')
print("")
print(hotels)

There are 462 different hotels in the dataset. 

['Rancho Valencia Resort Spa' 'Aloft Arundel Mills'
 'Hampton Inn Suites PortlandVancouver' 'Hotel Phillips'
 'The Inn at Solvang' 'Econolodge' 'Hotel Zelos' 'EconoLodge'
 'Hampton Inn Suites National HarborAlexandria Area'
 'Fairmont Grand Del Mar' 'Best Western at OHare'
 'Clarion Resort Waterpark' 'Virgin Hotels Chicago' 'The Boxer'
 'The Old No. 77 Hotel & Chandlery'
 'Main Street Station Casino Brewery Hotel' 'Fremont Hotel & Casino'
 'The Charlesmark Hotel' 'AC Hotel by Marriott Boston Downtown'
 'The Hotel of South Beach' 'The Pearl Hotel' 'Hotel Renew'
 'Four Seasons Hotel New York Downtown' 'The James New York ‚Äì NoMad'
 'MOXY New Orleans Downtown/French Quarter Area' 'Bienville House'
 'Room Mate Lord Balfour' 'The Westin Las Vegas Hotel & Spa' 'The Broome'
 'W Boston' 'Mandarin Oriental, Miami' 'The Ritz-Carlton, Boston'
 'The St. Regis New York' 'Miyako Hotel Los Angeles'
 'AC Hotel Chicago Downtown' 'Ace Hotel Chicago' 'dan

In [11]:
#Create new data frame to hold name, city and review text
groupByNameAndCity = data[['name', 'city', 'province', 'reviews.text']]

#Concatenate review texts for each group and create a new column all_reviews.  
groupByNameAndCity['all_reviews'] = groupByNameAndCity.groupby(['name', 'city'])['reviews.text'].transform(lambda review: ' '.join(review))

#Cleanup, drop old review column, then remove duplicates. 
groupByNameAndCity = groupByNameAndCity.drop(columns=['reviews.text'])
groupByNameAndCity = groupByNameAndCity.drop_duplicates()

#Preview, now we have one row per hote-city with all reviews in one field. 
groupByNameAndCity.head(20)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,name,city,province,all_reviews
0,Rancho Valencia Resort Spa,Rancho Santa Fe,CA,Our experience at Rancho Valencia was absolute...
3,Aloft Arundel Mills,Hanover,MD,Currently in bed writing this for the past hr ...
9,Hampton Inn Suites PortlandVancouver,Vancouver,WA,"In my line of work, I use meeting space in hot..."
15,Hotel Phillips,Kansas City,MO,Old hotel with many remaining architectural ch...
20,The Inn at Solvang,Huntingdon,PA,Everything was sold out in State College for G...
21,Econolodge,Perry,GA,I work here in Perry about 6 days out of the w...
22,Hotel Zelos,San Francisco,CA,The hotel is just one of many that are on this...
26,EconoLodge,Vineland,NJ,After getting the bait and switch I decided I'...
28,Hampton Inn Suites National HarborAlexandria Area,Oxon Hill,MD,Hotel is in the perfect spot at the perfect pr...
59,Fairmont Grand Del Mar,San Diego,CA,We hosted our annual Client Event at The Grand...


In [12]:
groupByNameAndCity.shape

(478, 4)

In [13]:
#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Construct the required TF-IDF matrix by fitting and transforming the reviews_text. 
tfidf_matrix = tfidf.fit_transform(groupByNameAndCity['all_reviews'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(478, 10522)

In [14]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix) 
cosine_sim

array([[1.        , 0.05863792, 0.07082922, ..., 0.09441375, 0.04775548,
        0.01903278],
       [0.05863792, 1.        , 0.12855056, ..., 0.18870781, 0.10115468,
        0.03092227],
       [0.07082922, 0.12855056, 1.        , ..., 0.24625571, 0.15112957,
        0.02266315],
       ...,
       [0.09441375, 0.18870781, 0.24625571, ..., 1.        , 0.17964553,
        0.04426755],
       [0.04775548, 0.10115468, 0.15112957, ..., 0.17964553, 1.        ,
        0.01333912],
       [0.01903278, 0.03092227, 0.02266315, ..., 0.04426755, 0.01333912,
        1.        ]])

In [15]:
#Turn the cosine_sim matrix into a DataFrame.    
pd.DataFrame(cosine_sim, columns=groupByNameAndCity.name.values, index=groupByNameAndCity.name.values)

,Rancho Valencia Resort Spa,Aloft Arundel Mills,Hampton Inn Suites PortlandVancouver,Hotel Phillips,The Inn at Solvang,Econolodge,Hotel Zelos,EconoLodge,Hampton Inn Suites National HarborAlexandria Area,Fairmont Grand Del Mar,Best Western at OHare,Clarion Resort Waterpark,Virgin Hotels Chicago,The Boxer,The Old No. 77 Hotel & Chandlery,Main Street Station Casino Brewery Hotel,Fremont Hotel & Casino,The Charlesmark Hotel,AC Hotel by Marriott Boston Downtown,The Hotel of South Beach,The Pearl Hotel,Hotel Renew,Four Seasons Hotel New York Downtown,The James New York ‚Äì NoMad,MOXY New Orleans Downtown/French Quarter Area,Bienville House,Room Mate Lord Balfour,The Westin Las Vegas Hotel & Spa,The Broome,W Boston,"Mandarin Oriental, Miami","The Ritz-Carlton, Boston",The St. Regis New York,Miyako Hotel Los Angeles,AC Hotel Chicago Downtown,Ace Hotel Chicago,dana hotel and spa,Kinzie Hotel,Moxy Chicago Downtown,Montage Beverly Hills,...,Honua Kai Resort Spa,Virginia Beach Resort Hotel Conference Center,Hilton Baltimore BWI Airport,Staybridge Suites Lincoln Northeast,Hyatt Place Baltimore/Inner Harbor,Holiday Inn Sarasota-Airport,Baymont Inn Suites Grand Rapids SW/Byron Center,Hampton Inn and Suites Baltimore Inner Harbor,Hampton Inn and Suites Williamsburg-Central,"Pier 5 Hotel Baltimore, Curio Collection by Hilton",Homewood Suites by Hilton Baltimore,SpringHill Suites by Marriott Virginia Beach Oceanfront,Sense Beach House,Home2 Suites by Hilton Baltimore Downtown,Sheraton Portland Airport Hotel,Fairfield Inn and Suites by Marriott Virginia Beach Oceanfront,Villa Paradiso,Motel 6 Grand Rapids - Airport,La Quinta Inn Suites Warner Robins - Robins AFB,Best Western Plus Lockport Hotel,Blue Sky Lodge,Hilton Garden Inn,Courtyard Austin South,The Star Inn,Hyatt Place Atlanta Airport-South,Table Mountain Inn,Quality Inn,Comfort Inn,Sonesta ES Suites Flagstaff,Aqua Soleil Hotel And Mineral Water Spa,Motel 6,Best Western Plus Arroyo Roble Hotel & Creekside Villas,hotel le bleu,Residence Inn Dallas Richardson,Homewood Suites by Hilton Orlando-Nearest to Univ Studios,Tradewinds Restaurant,Hampton Inn Union City,Embassy Suites by Hilton Baltimore at BWI Airport,Four Points by Sheraton Plainview Long Island,The Resort at Longboat Key Club
Rancho Valencia Resort Spa,1.000000,0.058638,0.070829,0.037516,0.045586,0.011182,0.066820,0.014319,0.083935,0.107467,0.038261,0.011033,0.107655,0.090113,0.054096,0.056408,0.057704,0.073283,0.024933,0.057097,0.030454,0.111527,0.026284,0.010869,0.058836,0.024177,0.060810,0.054399,0.056946,0.066401,0.065035,0.021261,0.091554,0.038207,0.042830,0.020986,0.074557,0.076543,0.051690,0.053696,...,0.092324,0.085629,0.086347,0.033181,0.090835,0.009871,0.016962,0.045905,0.033805,0.081770,0.045367,0.057963,0.066217,0.005856,0.043893,0.048074,0.050144,0.019756,0.026594,0.054049,0.025099,0.010826,0.023754,0.014661,0.095082,0.034362,0.039391,0.006720,0.067318,0.000000,0.092644,0.079214,0.043275,0.011027,0.047971,0.114275,0.061830,0.094414,0.047755,0.019033
Aloft Arundel Mills,0.058638,1.000000,0.128551,0.098435,0.016799,0.100116,0.096792,0.044097,0.131632,0.086040,0.035426,0.009640,0.101130,0.118113,0.089742,0.069025,0.089958,0.118906,0.044596,0.095580,0.060150,0.165201,0.027914,0.044604,0.066826,0.010863,0.096363,0.099117,0.055310,0.120216,0.068848,0.054824,0.108402,0.066737,0.090075,0.053065,0.113553,0.137664,0.039898,0.073310,...,0.085448,0.150942,0.192295,0.094170,0.148928,0.008269,0.030115,0.112515,0.073740,0.168388,0.093402,0.122346,0.100178,0.047232,0.071924,0.077228,0.086747,0.040541,0.087922,0.089349,0.090423,0.025378,0.055254,0.061612,0.146766,0.055157,0.056824,0.074758,0.117119,0.020178,0.034896,0.109503,0.090447,0.043718,0.030790,0.114990,0.083293,0.188708,0.101155,0.030922
Hampton Inn Suites PortlandVancouver,0.070829,0.128551,1.000000,0.169071,0.042403,0.071689,0.155585,0.025205,0.249031,0.157509,0.099797,0.040798,0.212058,0.232853,0.169592,0.121712,0.169353,0.268751,0.085406,0.151158,0.078616,0.230086,0.085370,0

In [16]:
#A list of hotel names and their associated index. 
indices = pd.Series(groupByNameAndCity.index, index=groupByNameAndCity['name']).drop_duplicates()
indices[0:10] 

name
Rancho Valencia Resort Spa                            0
Aloft Arundel Mills                                   3
Hampton Inn Suites PortlandVancouver                  9
Hotel Phillips                                       15
The Inn at Solvang                                   20
Econolodge                                           21
Hotel Zelos                                          22
EconoLodge                                           26
Hampton Inn Suites National HarborAlexandria Area    28
Fairmont Grand Del Mar                               59
dtype: int64

In [17]:
#I will pick "Rancho Valencia Resort Spa" and see what are the most similar hotels based on the review text. 
idx = indices['Rancho Valencia Resort Spa']
idx 

0

In [18]:
# Get the pairwise similarity scores of all hotels with that hotel. 
list(enumerate(cosine_sim[idx]))
sim_scores = list(enumerate(cosine_sim[idx]))
len(sim_scores) 

478

In [19]:
# Sort the hotels based on the similarity scores. 
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True) 
sim_scores[1:11]

[(168, 0.13169281011043593),
 (356, 0.12409740776070381),
 (95, 0.12213515092504901),
 (197, 0.11942974982092452),
 (182, 0.11623504655142164),
 (473, 0.1142745361035582),
 (21, 0.11152653452554154),
 (12, 0.10765527764120346),
 (9, 0.10746663499536074),
 (385, 0.10652600977630781)]

In [20]:
# Get the scores of the 10 most similar hotels
sim_scores = sim_scores[1:11]
sim_scores

[(168, 0.13169281011043593),
 (356, 0.12409740776070381),
 (95, 0.12213515092504901),
 (197, 0.11942974982092452),
 (182, 0.11623504655142164),
 (473, 0.1142745361035582),
 (21, 0.11152653452554154),
 (12, 0.10765527764120346),
 (9, 0.10746663499536074),
 (385, 0.10652600977630781)]

In [22]:
# Get the hotel indices.  
hotel_indices = []
for id, similarity in sim_scores: 
  hotel_indices.append(id) 

hotel_indices

[168, 356, 95, 197, 182, 473, 21, 12, 9, 385]

In [23]:
# Return the top 10 most similar hotels.  
top10_hotel_names = groupByNameAndCity[['name', 'city', 'province']].iloc[hotel_indices]
top10_hotel_names

,name,city,province
2115,Hyatt Regency Grand Cypress,Orlando,FL
2909,The Varden Hotel,Long Beach,CA
1516,Cornstalk Hotel,New Orleans,LA
2397,Hyatt Regency-valencia,Valencia,CA
2308,Omni Royal Orleans,New Orleans,LA
3921,Tradewinds Restaurant,Orlando,FL
376,Hotel Renew,Honolulu,HI
80,Virgin Hotels Chicago,Chicago,IL
59,Fairmont Grand Del Mar,San Diego,CA
3011,Seven Gables Inn,Saint Louis,MO
